# Regression Uncertainty
> "Learning uncertainty for regression using higher order distribution"

- toc: false
- branch: master
- badges: true
- comments: true
- image: images/nig_regression.png
- categories: [probability, regression, uncertainty]
